In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import torch
import math
import pickle


In [32]:
#split the dataset

dataset = pickle.load(open('datasets.pickle','rb'))

batchsize = 256
#train_x,test_x,train_y,test_y = train_test_split(torch.tensor(X),torch.tensor(Y),test_size=0.2,stratify=torch.tensor(Y))
#train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2,stratify=train_y)
#trainset = torch.utils.data.TensorDataset(train_x,train_y)
trainset = dataset[0]
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batchsize,shuffle=True)

#valset = torch.utils.data.TensorDataset(val_x,val_y)
valset = dataset[1]
valloader = torch.utils.data.DataLoader(valset,batch_size=batchsize,shuffle=True)

#testset = torch.utils.data.TensorDataset(test_x,test_y)

testset = dataset[2]
testloader = torch.utils.data.DataLoader(testset,batch_size=batchsize,shuffle=True)



print(trainset[0][0][:4],trainset[0][1].shape)

input_dim = 4
output_dim = trainset[0][1].shape[0]

tensor([ 209, 5287,    5,    2]) torch.Size([5911])


In [117]:

#define the model

class MLP(torch.nn.Module):

	def __init__(self, in_channels, hidden_channels, out_channels):
		super().__init__()
		self.emb1 = torch.nn.Embedding(100000,32)
		self.emb2 = torch.nn.Embedding(100000,32)
		self.timeday = torch.nn.Linear(2,32)
		self.class1 = torch.nn.Linear(96, hidden_channels)
		self.class2 = torch.nn.Linear(hidden_channels, 32)
		self.class3 = torch.nn.Linear(32,16)
		self.class4 = torch.nn.Linear(16, out_channels)



	def forward(self,data):


		user = self.emb1(data[:,0])
		loc = self.emb2(data[:, 1])
		#print(user.shape,loc.shape)
		hour = data[:,2]
		day = data[:,3]
		timeday = torch.stack([hour,day],dim=-1)
		#print(timeday.shape)
		timeday = self.timeday(timeday.float())
		x = torch.cat((user,loc,timeday),dim=1)
		#print(x.shape)
		x = self.class1(x).relu()
		x = self.class2(x).relu()
		x = self.class3(x).relu()
		x = self.class4(x)

		return x

In [118]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

lr = 0.001
wd = 5e-4

model = MLP(4,64,output_dim).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr, weight_decay=wd)
criterion = torch.nn.CrossEntropyLoss()

print(model)


Using device: cuda
MLP(
  (emb1): Embedding(100000, 32)
  (emb2): Embedding(100000, 32)
  (timeday): Linear(in_features=2, out_features=32, bias=True)
  (class1): Linear(in_features=96, out_features=64, bias=True)
  (class2): Linear(in_features=64, out_features=32, bias=True)
  (class3): Linear(in_features=32, out_features=16, bias=True)
  (class4): Linear(in_features=16, out_features=5911, bias=True)
)


In [120]:
import tqdm
numepoch = 500

#training phase

with tqdm.tqdm(total=numepoch*len(trainloader), unit='batches') as pbar:
  losses = []
  for epoch in range(numepoch):

    model.train()
    optimizer.zero_grad()

    print(f"EPOCH = {epoch}")

    for batch in trainloader:

      input = batch[0].to(device)
      label = batch[1].to(device)

      #print(input.shape)
      #print(label.shape)
      #one_hot_encode_labels(label)
      #print(input.shape,input[:,:4].shape)

      out = model(input[:,:4])

      #print(out.shape)
      #out = model(input)
      #out=out.squeeze(dim=1)
      #print(out)


      loss = criterion(out,label)
      losses.append(loss)
      loss.backward()
      optimizer.step()



      pbar.set_postfix({'Loss': loss.item()})
      pbar.update()


      #print(df)


  0%|          | 6/1500 [00:00<00:51, 29.07batches/s, Loss=2]   

EPOCH = 0
EPOCH = 1
EPOCH = 2


  1%|▏         | 22/1500 [00:00<00:29, 50.21batches/s, Loss=1.64]

EPOCH = 3
EPOCH = 4
EPOCH = 5
EPOCH = 6
EPOCH = 7


  3%|▎         | 42/1500 [00:00<00:18, 79.21batches/s, Loss=1.21]

...

EPOCH = 483
EPOCH = 484
EPOCH = 485
EPOCH = 486
EPOCH = 487
EPOCH = 488


 99%|█████████▉| 1485/1500 [00:17<00:00, 92.06batches/s, Loss=0.0143]

EPOCH = 489
EPOCH = 490
EPOCH = 491
EPOCH = 492
EPOCH = 493
EPOCH = 494
EPOCH = 495


100%|██████████| 1500/1500 [00:17<00:00, 85.03batches/s, Loss=0.0143]

EPOCH = 496
EPOCH = 497
EPOCH = 498
EPOCH = 499


In [ ]:
import matplotlib.pyplot as plt
#print CE loss evolution
ind = [i for i in range(len(losses))]
losses = [loss.cpu().detach().numpy() for loss in losses]
plt.plot(ind,losses)
plt.show()

In [124]:
import tqdm

criterion = torch.nn.CrossEntropyLoss()

#testing phase

with tqdm.tqdm(total=len(testloader), unit='batches') as pbar:
  losses = []

  model.eval()

  for batch in testloader:

    input = batch[0].to(device)
    label = batch[1].to(device)

    out = model(input[:,:4])

    loss = criterion(out,label)

    losses.append(loss)

    pbar.set_postfix({'Loss': loss.item()})
    pbar.update()

print(sum(losses)/len(losses))


100%|██████████| 1/1 [00:00<00:00, 107.00batches/s, Loss=40]

tensor(40.0012, device='cuda:0', grad_fn=<DivBackward0>)


In [ ]:
import matplotlib.pyplot as plt

#print loss evolution

ind = [i for i in range(len(losses))]
losses = [loss.cpu().detach().numpy() for loss in losses]
plt.plot(ind,losses)
plt.show()